Background:  
I want to open a "new restaurant for kids in gangnam district in seoul, south korea". so I will find the place where many kids are playing . I will find the place where schools are crowed in gangnam district.

2.A description of the data and how it will be used to solve the problem.

Data: latitude and longitude data of schools in gangnam district in seoul, south korea. I will use four square's location data

How: 
       (1) make ten condidate points (randomly chosen) in gangnam distruct.
       (2) make 1 kilometer radius circle for each candidate point.
       (3) calcuate how many schools are in each circle.
       (4) find the circle which contains maximum schools. the center of that circle will be the best place for the restaurant.

# week5
# A full report consisting of all of the following components 

(0) purpose: what is the best places for kid restraurant in kangnam district in seoul, in korea.

(1)Introduction
 -I am living in kangnam district in seoul, korea. one of my neightbor was interested in kid restaurant business. so i prepared this project for him.    

(2)data where you describe the data that will be used to solve the problem and the source of the data.
 -I  used foursqure's geographic data to find places where schools are concentrated.  

(3)Methodology section 
 I used brute force algorithm to find best place.
 firstly, I selected 25 candidate places randomly.
 and then used foursqure's API(api.foursquare.com/v2/venues/search) to calculated how many schools are close to candidate places.
 I could find best places (the place where most schools are close to)  
 
(4) Results section where you discuss the results.
 -two places are good.  10 schools are close to them(within 500m)
place0 latitude: 37.517886439865514   longtude: 127.0263715192932
place1 latitude: 37.51719236175003    longtude: 127.02631642490377

(5) Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
 -Conclusion section where you conclude the report.
foursqure's API is easy but strong for the spatial analysis. so i can find best places for kid restaurant very efficiently.   

In [1]:
#week5 codes
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import random  

print('Folium installed')
print('Libraries imported.')

'conda'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'conda'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


Folium installed
Libraries imported.


In [2]:
#prepare repeated query data
CLIENT_ID = '1WCWYWYW2VNJS4K4WFQBDEAWQ5VCFDZKHJAAZZJ3EFFYIMIX' # your Foursquare ID
CLIENT_SECRET = '5THR0KX03UUSP12T0IAX5UMHEKCVDHEPGO4FQF04PPUQ41IN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
client_id=CLIENT_ID
client_secret=CLIENT_SECRET
version=VERSION
limit=LIMIT

Your credentails:
CLIENT_ID: 1WCWYWYW2VNJS4K4WFQBDEAWQ5VCFDZKHJAAZZJ3EFFYIMIX
CLIENT_SECRET:5THR0KX03UUSP12T0IAX5UMHEKCVDHEPGO4FQF04PPUQ41IN


In [3]:
#prepare latitude and longitude data for 25 (randomly choosen) restaurant candidates
latitude=[]
longitude=[]
lat_start = 37.5207
lon_start = 127.0260
for i in range(0,5):
    for j in range(0,5):
        latitude.append( lat_start-i*0.001+random.random()*0.0005 ) 
        longitude.append(lon_start+j*0.002+random.random()*0.0005 )

total_size = len(latitude)     
print(total_size)

25


In [4]:
#keyword is school 
search_query = 'school'
#search diatemter is 1 kilometer
radius = 500
print(search_query + ' .... OK!')

school .... OK!


In [5]:
#make url for each restaurant candidate
url=[]
for i in range(0,total_size):
    url.append(  'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude[i], longitude[i], VERSION, search_query, radius, LIMIT))


In [6]:
#calculate how many schools are within 1km diameter of restanrant candidates
results=[]
max_value=0
max_value_index=[0]
for i in range(0, total_size):
    results.append(  requests.get(url[i]).json()  )
    venues = results[i]['response']['venues']
    venues
    dataframe = json_normalize(venues)
    dataframe.head()
    if( max_value<len(dataframe.index) ):
        max_value = len(dataframe.index)
        del max_value_index
        max_value_index=[]
        max_value_index.append(i)
#        print("reset index="+ str(i))
    elif (max_value==len(dataframe.index)):
        max_value_index.append(i)        
    print ( 'schools='+str( len(dataframe.index)) + "      candidate latitude:" + str( latitude[i]) + " longitude:" + str( longitude[i]) )

schools=8      candidate latitude:37.5209952281683 longitude:127.02619579562439
schools=6      candidate latitude:37.52116405528831 longitude:127.02837317184218
schools=3      candidate latitude:37.52100012495459 longitude:127.03043281533131
schools=4      candidate latitude:37.52103060857341 longitude:127.0322884138989
schools=7      candidate latitude:37.520957354282274 longitude:127.03445174935595
schools=8      candidate latitude:37.51985050163391 longitude:127.02628941505938
schools=6      candidate latitude:37.519769917014436 longitude:127.02809060783707
schools=5      candidate latitude:37.51993491277166 longitude:127.03025011679968
schools=5      candidate latitude:37.51985552587978 longitude:127.03220391377761
schools=4      candidate latitude:37.51984232311199 longitude:127.03412850635034
schools=8      candidate latitude:37.51907559906177 longitude:127.02648152171781
schools=7      candidate latitude:37.518737385505254 longitude:127.0281725303226
schools=4      candidate lat

In [7]:
#final result!!!
#find best places for kid restaurant
print('max_value='+str(max_value)+'  best places:' +str( len(max_value_index))  )
for i in range(0,len(max_value_index) ) :
    print( str ( latitude[ max_value_index[i]])+ "   "+ str ( longitude[ max_value_index[i]])  )

max_value=12  best places:1
37.51676335096894   127.0260185476052
